# Two layer net

## 背景

为了提高预测准确率，我们需要使用多层神经网络，因为一般来说网络的层数越多其表达能力越强，因为其参数更多，能表达出的状态信息更多，所以表达能力越强。多层神经网络可以应对更加复杂的问题，这一节我们先从一个小例子开始：构建一个两层神经网络，这一节我们构建的简单两层神经网络可以达到51%的准确率。

## 引入依赖

In [1]:
import $plugin.$ivy.`com.thoughtworks.implicit-dependent-type::implicit-dependent-type:2.0.0`

import $ivy.`com.thoughtworks.deeplearning::differentiableany:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiablenothing:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiableseq:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiabledouble:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiablefloat:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiablehlist:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiablecoproduct:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiableindarray:1.0.0-RC7`
import $ivy.`org.rauschig:jarchivelib:0.5.0`

import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.0`

import java.io.{FileInputStream, InputStream}


import com.thoughtworks.deeplearning
import org.nd4j.linalg.api.ndarray.INDArray
import com.thoughtworks.deeplearning.DifferentiableHList._
import com.thoughtworks.deeplearning.DifferentiableDouble._
import com.thoughtworks.deeplearning.DifferentiableINDArray._
import com.thoughtworks.deeplearning.DifferentiableAny._
import com.thoughtworks.deeplearning.DifferentiableINDArray.Optimizers._
import com.thoughtworks.deeplearning.DifferentiableINDArray.Layers.Weight
import com.thoughtworks.deeplearning.{
  DifferentiableHList,
  DifferentiableINDArray,
  Layer,
  Symbolic
}
import com.thoughtworks.deeplearning.Layer.Batch
import com.thoughtworks.deeplearning.Symbolic.Layers.Identity
import com.thoughtworks.deeplearning.Symbolic._
import com.thoughtworks.deeplearning.Poly.MathFunctions._
import com.thoughtworks.deeplearning.Poly.MathMethods./
import com.thoughtworks.deeplearning.Poly.MathOps
import org.nd4j.linalg.api.ndarray.INDArray
import org.nd4j.linalg.cpu.nativecpu.NDArray
import org.nd4j.linalg.factory.Nd4j
import org.nd4j.linalg.indexing.{INDArrayIndex, NDArrayIndex}
import org.nd4j.linalg.ops.transforms.Transforms
import org.nd4s.Implicits._
import shapeless._

import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._

import scala.collection.immutable.IndexedSeq
import scala.util.Random

pprintConfig() = pprintConfig().copy(height = 2)//减少输出的行数，避免页面输出太长

import $file.ReadCIFAR10ToNDArray,ReadCIFAR10ToNDArray._
import $file.Utils,Utils._

import $plugin.$                                                                             


import $ivy.$                                                           

import $ivy.$                                                               

import $ivy.$                                                           

import $ivy.$                                                              

import $ivy.$                                                             

import $ivy.$                                                             

import $ivy.$                                                                 

import $ivy.$                                                                

import $ivy.$                               


import $ivy.$                                             


import java.io.{FileInputStream, InputStream}



import com.thoughtworks.deeplearning

import org.nd4j.linalg.api.ndarray.INDArray

import com.thoughtworks.deeplearning.Differentiabl

## 读取和处理数据

类似[前一节](https://thoughtworksinc.github.io/DeepLearning.scala/demo/MiniBatchGradientDescent.html)从CIFAR10 database中读取和处理测试数据的图片和标签信息

In [2]:
//CIFAR10中的图片共有10个分类(airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck)
val NumberOfClasses: Int = 10

//加载测试数据，我们读取100条作为测试数据
val testNDArray =
  ReadCIFAR10ToNDArray.readFromResource("/cifar-10-batches-bin/test_batch.bin", 100)

val testData = testNDArray.head

val testExpectResult = testNDArray.tail.head

val vectorizedTestExpectResult = Utils.makeVectorized(testExpectResult, NumberOfClasses)

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


NumberOfClasses: Int = 10
testNDArray: INDArray :: INDArray :: HNil = [[0.62, 0.62, 0.64, 0.65, 0.62, 0.61, 0.63, 0.62, 0.62, 0.62, 0.63, 0.62, 0.63, 0.65, 0.66, 0.66, 0.65, 0.63, 0.62, 0.62, 0.61, 0.58, 0.59, 0.58, 0.58, 0.56, 0....
testData: INDArray = [[0.62, 0.62, 0.64, 0.65, 0.62, 0.61, 0.63, 0.62, 0.62, 0.62, 0.63, 0.62, 0.63, 0.65, 0.66, 0.66, 0.65, 0.63, 0.62, 0.62, 0.61, 0.58, 0.59, 0.58, 0.58, 0.56, 0....
testExpectResult: INDArray = [3.00, 8.00, 8.00, 0.00, 6.00, 6.00, 1.00, 6.00, 3.00, 1.00, 0.00, 9.00, 5.00, 7.00, 9.00, 8.00, 5.00, 7.00, 8.00, 6.00, 7.00, 0.00, 4.00, 9.00, 5.00, 2.00, 4.0...
vectorizedTestExpectResult: INDArray = [[0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
 [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00],
...

## 构建神经网络

### 参数调优

跟前一节不同，这节我们假如一些参数调优的手段，设置学习率和使用[L2Regularization](https://en.wikipedia.org/wiki/Regularization_(mathematics)),L2Regularization可以用来避免过拟合。我们还使用了每个迭代learningRate都下降为原来的0.9995倍的办法来解决训练时间增长时因为learningRate相对太大导致loss下降太慢或者不下降的问题。

In [3]:
implicit val optimizerFactory = new DifferentiableINDArray.OptimizerFactory {
  override def ndArrayOptimizer(weight: Weight): Optimizer = {
    new LearningRate with L2Regularization {

      var learningRate = 0.001

      override protected def currentLearningRate(): Double = {
        learningRate *= 0.9995
        learningRate
      }

      override protected def l2Regularization: Double = 0.03
    }
  }
}

optimizerFactory: AnyRef with OptimizerFactory = $sess.cmd2Wrapper$Helper$$anon$2@28287c5c

### 编写第一层神经网络

这是全连接和[relu](http://stats.stackexchange.com/questions/126238/what-are-the-advantages-of-relu-over-sigmoid-function-in-deep-neural-network)组成的神经网络。

In [4]:
def fullyConnectedThenRelu(inputSize: Int, outputSize: Int)(
    implicit row: INDArray @Symbolic): INDArray @Symbolic = {
  val w = (Nd4j.randn(inputSize, outputSize) / math.sqrt(outputSize / 2.0)).toWeight * 0.1
  val b = Nd4j.zeros(outputSize).toWeight
  max((row dot w) + b, 0.0)
}

defined function fullyConnectedThenRelu

### 编写第二层神经网络

跟上节相同，我们使用softmax作为分类器

In [5]:
def softmax(implicit scores: INDArray @Symbolic): INDArray @Symbolic = {
  val expScores = exp(scores)
  expScores / expScores.sum(1)
}

defined function softmax

编写两层神经网络的第二层神经网络，这时一层全连接一层softmax的神经网络。

In [6]:
def fullyConnectedThenSoftmax(inputSize: Int, outputSize: Int)(
    implicit row: INDArray @Symbolic): INDArray @Symbolic = {
  val w = (Nd4j.randn(inputSize, outputSize) / math.sqrt(outputSize)).toWeight //* 0.1
  val b = Nd4j.zeros(outputSize).toWeight
  softmax.compose((row dot w) + b)
}

defined function fullyConnectedThenSoftmax

### 组合两层神经网络

为了实现两层神经网络我们使用`compose`将上面的两层神经网络组合起来，组成一个两层神经网络。`a.compose(b)`可以将`b`的输出作为`a`输入从而将两层神经网络组合起来。

In [7]:
def hiddenLayer(implicit input: INDArray @Symbolic): INDArray @Symbolic = {
  val layer0 = fullyConnectedThenRelu(3072, 500).compose(input)
  fullyConnectedThenSoftmax(500, 10).compose(layer0)
}

val predictor = hiddenLayer

defined function hiddenLayer
predictor: (Symbolic.To[INDArray]{type OutputData = org.nd4j.linalg.api.ndarray.INDArray;type OutputDelta = org.nd4j.linalg.api.ndarray.INDArray;type InputData = org.nd4j.linalg.api.ndarray.INDArray;type InputDelta = org.nd4j.linalg.api.ndarray.INDArray})#@ = Compose(Compose(MultiplyINDArray(Exp(Identity()),Reciprocal(Sum(Exp(Identity()),WrappedArray(1)))),PlusINDArray(Dot(Identity(),Weight([[-0.20, -0.33, -0.19, -0....

### 编写LossFunction并组合输入层和[隐含层](http://stats.stackexchange.com/questions/63152/what-does-the-hidden-layer-in-a-neural-network-compute)

In [8]:
def crossEntropy(
    implicit pair: (INDArray :: INDArray :: HNil) @Symbolic): Double @Symbolic = {
  val score = pair.head
  val label = pair.tail.head
  -(label * log(score * 0.9 + 0.1) + (1.0 - label) * log(1.0 - score * 0.9)).mean
}

def network(
   implicit pair: (INDArray :: INDArray :: HNil) @Symbolic): Double @Symbolic = {
  val input = pair.head
  val label = pair.tail.head
  val score: INDArray @Symbolic = predictor.compose(input)
  val hnilLayer: HNil @Symbolic = HNil
  crossEntropy.compose(score :: label :: hnilLayer)
}

val trainer = network

defined function crossEntropy
defined function network
trainer: (Symbolic.To[Double]{type OutputData = Double;type OutputDelta = Double;type InputData = shapeless.::[org.nd4j.linalg.api.ndarray.INDArray,shapeless.::[org.nd4j.linalg.api.ndarray.INDArray,shapeless.HNil]];type InputDelta = shapeless.:+:[org.nd4j.linalg.api.ndarray.INDArray,shapeless.:+:[org.nd4j.linalg.api.ndarray.INDArray,shapeless.CNil]]})#@ = Compose(Negative(ReduceMean(PlusINDArray(MultiplyINDArray(Head(Tail(Identity())),Log(PlusDouble(MultiplyDouble(Head(Identity()),Literal(0.9)),Literal(0.1)))),Mu...

## 训练神经网络

与上一节相同，训练神经网络并观察每次训练loss的变化

In [9]:
val random = new Random

val MiniBatchSize = 256


def trainData(randomIndexArray: Array[Int]): Double = {
  val trainNDArray :: expectLabel :: shapeless.HNil =
    ReadCIFAR10ToNDArray.getSGDTrainNDArray(randomIndexArray)
  val input =
    trainNDArray.reshape(MiniBatchSize, 3072)

  val expectLabelVectorized =
    Utils.makeVectorized(expectLabel, NumberOfClasses)
  trainer.train(input :: expectLabelVectorized :: HNil)
}

val lossSeq =
  (
    for (iteration <- 0 to 50) yield {
      val randomIndex = random
        .shuffle[Int, IndexedSeq](0 until 10000) //https://issues.scala-lang.org/browse/SI-6948
        .toArray
      for (times <- 0 until 10000 / MiniBatchSize) yield {
        val randomIndexArray =
          randomIndex.slice(times * MiniBatchSize,
                            (times + 1) * MiniBatchSize)
          val loss = trainData(randomIndexArray)
          if(times == 3 & iteration % 5 == 4){
            println("at epoch " + (iteration / 5 + 1) + " loss is :" + loss)
          }
          loss
      }
    }
  ).flatten

val plot = Seq(
  Scatter(lossSeq.indices, lossSeq)
)

plot.plot(
  title = "loss by time"
)

at epoch 1 loss is :0.21449708938598633
at epoch 2 loss is :0.20017881393432618
at epoch 3 loss is :0.19701696634292604
at epoch 4 loss is :0.19029536247253417
at epoch 5 loss is :0.20084950923919678
at epoch 6 loss is :0.18678001165390015
at epoch 7 loss is :0.18276526927947997
at epoch 8 loss is :0.17886807918548583
at epoch 9 loss is :0.19118783473968506
at epoch 10 loss is :0.17489395141601563


random: Random = scala.util.Random@454aadb9
MiniBatchSize: Int = 256
defined function trainData
lossSeq: IndexedSeq[Double] = Vector(
  0.25571749210357664,
...
plot: Seq[Scatter] = List(
  Scatter(
...
res8_5: String = "plot-514606496"

跟上一节相同，我们使用测试数据来查看神经网络判断结果并计算准确率。这次准确率应该会上升到51%左右。

In [10]:
val right = Utils.getAccuracy(predictor.predict(testData), testExpectResult)
println(s"the result is $right %")

the result is 46.0 %


right: Double = 46.0

## 总结

在这节中我们学到了：

* 参数调优
* L2Regularization
* Relu
* 构建一个两层神经网络


[完整代码](https://github.com/izhangzhihao/deeplearning-tutorial/blob/master/src/main/scala/com/thoughtworks/deeplearning/tutorial/TwoLayerNet.scala)